- Filename:     Enhance_Image_Lightning.ipynb
- Author:       Nabegh Sayegh
- Email:        Nabe8sa@gmail.com
- Created:      02.08.2025
- Last Modified: 04.08.2025
- Description:  Image processing pipeline that uses OpenCV for:
    - Brightness calculation via HSV conversion
    - Dark image enhancement using CLAHE in LAB color space
    - Bright image correction through gamma adjustment
    - Efficient batch processing with directory structure preservation

This code is for Google Colab to mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
 

Mounted at /content/drive


In [ ]:
!pip install tqdm

In [ ]:
input_root = "/content/drive/MyDrive/Yasmeen_Project/Input_Folder"
output_root = "/content/drive/MyDrive/Yasmeen_Project/Output_Folder"


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
def calculate_brightness(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return np.mean(hsv[:, :, 2])

def enhance_dark_image(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

def enhance_bright_image(image, gamma=0.6):
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [ ]:
def process_all_images(input_root, output_root):
    for root, dirs, files in os.walk(input_root):
        for file in tqdm(files):
            if not file.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            input_path = os.path.join(root, file)
            rel_path = os.path.relpath(input_path, input_root)
            output_path = os.path.join(output_root, rel_path)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            image = cv2.imread(input_path)
            if image is None:
                continue

            brightness = calculate_brightness(image)

            if brightness < 80:
                enhanced = enhance_dark_image(image)
            elif brightness > 200:
                enhanced = enhance_bright_image(image)
            else:
                enhanced = image

            cv2.imwrite(output_path, enhanced)

process_all_images(input_root, output_root)

0it [00:00, ?it/s]
100%|██████████| 2056/2056 [02:56<00:00, 11.65it/s]
